## steps
1. ✅Understand the whole picture
   - Quantify final objectives ?
   - What is the current solution (if any) ? Downsides ?
2. ✅Get data
3. ✅explore, visualize data => **insight**
4. ✅prepare data for machine learning algo
5. ✅select and train a model
6. fine-tune model <= **DOING**
   - Mesure the errors made by the model, with **RMSE** or **MAE**
7. present solution
8.  launch, monitor, maintain system

In [ ]:
import pandas as pd
housing = pd.read_feather("prepared-data/strat_train_set_features.feather").set_index("index")
housing_lables = (pd
    .read_feather("prepared-data/strat_train_set_lables.feather")
    .set_index("index")
    .median_house_value
)

# I. Grid Search

- Changing **Hyperparameters** with `GridSearchCV`
- Tell it the **Hyperparemeters**, and the range of **experiment values**

In [ ]:
from utils import preprocessing_end_part1
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

full_pipeline = Pipeline([
        ("preprocessing", preprocessing_end_part1),
        ("random_forest", RandomForestRegressor(random_state=42))
])
param_grid = [
    {
        'preprocessing__geo__n_clusters' : [5, 8, 10],
        'random_forest__max_features' : [4,6,8]
    },
    {
        'preprocessing__geo__n_clusters' : [10, 15],
        'random_forest__ max_features' : [6,8,10]
    }
]

grid_search = GridSearchCV(
    full_pipeline,
    param_grid=param_grid,
    cv=3,
    scoring='neg_root_mean_squared_error')

grid_search.fit(housing, housing_lables)

- pipeline, nested pipeline and parameters are *seperated* by **double under slashes '__'**:
  - `preprocessing__geo__n_clusters` tells `GridSearchCV` to change `n_clusters` parameter inside estimator **geo**, inside **preprocessing** step/transformer of the **Pipeline**
- `param_grid`:
  - first element tells `GridSearchCV` to try **3X3=9** combinations of **hyperparameters**
  - second element tells `GridSearchCV` to try **2X3=6** combinations of **hyperparameters**
  - => 15 Combinations of hyperparameters
- `cv = 3` tells the `grid_search` to perform crossvalidation on 3 folded dataset
- => The script will run **15** *(combinations of hyperparameters)* **X 3** *(3 folds)* = 45 rounds

- **NOTE TO SELF**: On this computer, in the part2 notebook, each `RandomForestRegressor` takes up 3 mins => **2hrs 15mins** to train 